In [26]:
import cvxpy as cp
import numpy as np
from cvxpylayers.torch import CvxpyLayer
import torch
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import time
import pandas as pd

In [27]:
seed = 17

sizes = [10,20,30,40,50]

num_constraints = 5

num_ellipsoid_defining_points = 4
# Number of opimisation steps:
epochs = 200


In [28]:
# Uncertain QCQP without context:

robust_optimal_value = []
robust_solving_times = []

deterministic_surrogate_best_value = []
deterministic_surrogate_solving_times = []
deterministic_surrogate_test_time = []

for n in sizes:
    
    print(f"Problem size {n} started at {time.asctime()}")
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    ## SOLVE PROBLEM USING ROBUST REFORMULATION
    ls = [n for nc in range(num_constraints)] # dimension of the multiplied axis for the SDM

    # Randomly generate the certain objective # cost vector
    c = np.random.rand(n,1)
    # Randomly generate constraints
    # uncertainty centre is [:][0]:
    # ellipsoid defining points are [1,...,num_constraints]
    
    Ps = {}
    bs = {}
    gammas = {}

    for nc in range(num_constraints):
        Ps[nc] = {}
        for i in range(num_ellipsoid_defining_points+1):
            Ps[nc][i] = np.random.rand(ls[nc], n)

        bs[nc] = {}
        for i in range(num_ellipsoid_defining_points+1):
            bs[nc][i] = np.random.rand(n,1)

        gammas[nc] = {}

        for i in range(num_ellipsoid_defining_points+1):
            gammas[nc][i] = np.random.rand(1)

        gammas[nc][0] += 1 # ensure problem is always feasible (regardless of init)

    ###
    #  SOLVE PROBLEM USING ROBUST REFORMULATION
    ###
    # For constraints of the type -x.T @ A_i.T @ A_i @ x + 2b_i.T @ x + gamma_i >= 0
    # and the uncertainty sets U_i = 
    # {(A_i,b_i,gamma_i) | (A[i][0],b[i][0],gamma[i][0]) + sum_j u(A[i][j],b[i][j],gamma[i][j]), ||u|| <= 1}
    # The following LMI is equivalent (Ben-Tal and Nemirovski, 1999)
    x = cp.Variable((n,1),name = "x")
    constraints = []
    LMI_vars = []
    lambdas = [cp.Variable(1, name = f"Lambda_{nc+1}") for nc in range(num_constraints)]
    for nc in range(num_constraints):
        # Create matrix variable which corresponds to the S-Lemma condition
        size_of_LMI = num_ellipsoid_defining_points + 1 + ls[nc]
        LMI_vars.append(cp.Variable((size_of_LMI,size_of_LMI),name = f"LMI_{nc}", symmetric = True))
        constraints += [LMI_vars[nc] >> 0]
        # Link matrix with conditions
        # [0,0] top left
        constraints += [LMI_vars[nc][0,0] == gammas[nc][0] + 2 * x.T @ bs[nc][0] - lambdas[nc]]

        # [0,1:num_ellipsoid_ellipsoid_defining_points+1] top row, num_ellipsoid_ellipsoid_defining_points entries after the first
        row = cp.hstack([(1/2)*gammas[nc][j] + x.T @ bs[nc][j] for j in range(1,num_ellipsoid_defining_points+1)])
        constraints += [LMI_vars[nc][[0],1:num_ellipsoid_defining_points+1] == row]

        # [0,1:num_ellipsoid_ellipsoid_defining_points+1] leftmost column, num_ellipsoid_ellipsoid_defining_points entries after the first
        # symmetric
        constraints += [LMI_vars[nc][1:num_ellipsoid_defining_points+1,[0]] == row.T]

        # [[i,i] for i in range(1,num_ellipsoid_defining_points+1)] diagonal lambdas
        for i in range(1,num_ellipsoid_defining_points+1):
            constraints += [LMI_vars[nc][i,i] == lambdas[nc]]
        # enforce zeros in the middle block off diagonal
        for i in range(1,num_ellipsoid_defining_points+1):
            for j in range(1,num_ellipsoid_defining_points+1):
                if i!= j:
                    constraints += [LMI_vars[nc][i,j] == 0]

        # Right most column set
        submatrix = cp.vstack([(Ps[nc][j] @ x).T for j in range(0, num_ellipsoid_defining_points + 1)])
        constraints += [LMI_vars[nc][:num_ellipsoid_defining_points+1,-ls[nc]:] == submatrix]

        # Bottom most row set, symmetric to above
        constraints += [LMI_vars[nc][-ls[nc]:,:num_ellipsoid_defining_points+1] == submatrix.T]

        # Bottom right most block matrix
        constraints += [LMI_vars[nc][-ls[nc]:,-ls[nc]:] == np.eye(ls[nc])]


    objective = cp.Minimize(c.T@x)
    problem = cp.Problem(objective,constraints)
    start = time.process_time()
    problem.solve()
    time_delta = time.process_time() - start
    print(problem.value, "robust reformulation solution")
    robust_optimal_value.append(problem.value)
    robust_solving_times.append(time_delta)
    
    ###
    #  SOLVE PROBLEM USING DETERMINISTIC SURROGATES
    ###
    
    ## Construct ROB_1 and ROB_2
    # outer problem
    x_det = cp.Variable((n,1), name= "decision")

    # Define parameters for problem (the variables of interest for optimisation)
    c_param = [cp.Parameter((n,1))]
    Ps_param = []
    bs_param = []
    gammas_param = []
    for nc in range(num_constraints):
        bs_param.append(cp.Parameter((n,1)))
        gammas_param.append(cp.Parameter(1))
        Ps_param.append(cp.Parameter((n,ls[nc])))

    # construct problem
    constraints = []
    for nc in range(num_constraints):
        constraints += [-cp.sum_squares(Ps_param[nc]@x_det) + 2*bs_param[nc].T@x_det + gammas_param[nc] >=0]

    objective = cp.Minimize(c_param[0].T@x_det)

    outer_problem = cp.Problem(objective,constraints)

    ROB_1 = CvxpyLayer(outer_problem, parameters=c_param + Ps_param + bs_param + gammas_param, variables=[x_det])


    # Inner problem
    # worst case solve

    x_fixed = cp.Parameter((n,1))

    constraints = []
    LMI_vars = []
    lambdas = [cp.Variable(1, name = f"Lambda_{nc+1}") for nc in range(num_constraints)]
    for nc in range(num_constraints):
        # Create matrix variable which corresponds to the S-Lemma condition
        size_of_LMI = num_ellipsoid_defining_points + 1 + ls[nc]
        LMI_vars.append(cp.Variable((size_of_LMI,size_of_LMI),name = f"LMI_{nc}", symmetric = True))
        # Link matrix with conditions
        # [0,0] top left
        constraints += [LMI_vars[nc][0,0] == gammas[nc][0] + 2 * x_fixed.T @ bs[nc][0] - lambdas[nc]]

        # [0,1:num_ellipsoid_ellipsoid_defining_points+1] top row, num_ellipsoid_ellipsoid_defining_points entries after the first
        row = cp.hstack([(1/2)*gammas[nc][j] + x_fixed.T @ bs[nc][j] for j in range(1,num_ellipsoid_defining_points+1)])
        constraints += [LMI_vars[nc][[0],1:num_ellipsoid_defining_points+1] == row]

        # [0,1:num_ellipsoid_ellipsoid_defining_points+1] leftmost column, num_ellipsoid_ellipsoid_defining_points entries after the first
        # symmetric
        constraints += [LMI_vars[nc][1:num_ellipsoid_defining_points+1,[0]] == row.T]

        # [[i,i] for i in range(1,num_ellipsoid_defining_points+1)] diagonal lambdas
        for i in range(1,num_ellipsoid_defining_points+1):
            constraints += [LMI_vars[nc][i,i] == lambdas[nc]]
        # enforce zeros in the middle block off diagonal
        for i in range(1,num_ellipsoid_defining_points+1):
            for j in range(1,num_ellipsoid_defining_points+1):
                if i!= j:
                    constraints += [LMI_vars[nc][i,j] == 0]

        # Right most column set
        submatrix = cp.vstack([(Ps[nc][j] @ x_fixed).T for j in range(0, num_ellipsoid_defining_points + 1)])
        constraints += [LMI_vars[nc][:num_ellipsoid_defining_points+1,-ls[nc]:] == submatrix]

        # Bottom most row set, symmetric to above
        constraints += [LMI_vars[nc][-ls[nc]:,:num_ellipsoid_defining_points+1] == submatrix.T]

        # Bottom right most block matrix
        constraints += [LMI_vars[nc][-ls[nc]:,-ls[nc]:] == np.eye(ls[nc])]


    objective_function = []
    multipliers = [1 for nc in range(num_constraints)]
    objective_function += [multipliers[nc]*cp.lambda_min(LMI_vars[nc]) for nc in range(num_constraints)]
    objective_function = cp.sum(objective_function)
    objective = cp.Maximize(objective_function)

    inner_problem = cp.Problem(objective, constraints)

    inner_vars_list = [lambdas,LMI_vars]
    inner_vars = [inner_vars_list[v][nc] for nc in range(num_constraints) for v in range(len(inner_vars_list))] # sort so iterated by constraint
    ROB_2 = CvxpyLayer(inner_problem, parameters= [x_fixed], variables=inner_vars)


    # initialise the parameters describing the deterministic problem randomly
    # In this case we are assuming there is no context
    pred_c = [torch.rand(n,1)]
    pred_A = []
    pred_b = []
    pred_gamma = []

    for nc in range(num_constraints):
        pred_A.append(torch.rand(ls[nc], n))
        pred_b.append(torch.rand(n,1))
        pred_gamma.append(torch.rand(1) + 1) # ensure feasibility

    torch_parameters_o = pred_c + pred_A + pred_b + pred_gamma

    torch_parameters = [torch.nn.Parameter(parameter.clone()) for parameter in torch_parameters_o]

    optimiser = torch.optim.Adam(
                torch_parameters,
                lr=0.1,
            )

    losses = []
    
    last_feasible_objective = np.inf
    
    start = time.process_time()
    for j in tqdm(range(epochs)):

        optimiser.zero_grad()

        loss = 0

        # Forward pass
        x_opt, = ROB_1(*(torch_parameters))
        
        # Add the objective value to the loss
        loss_p = torch.tensor(c.T).float()@x_opt
        loss += loss_p 

        # Solve ROB_2
        # exception: if problem returns unbounded the solution is feasible and we do not have to pass gradients for constraints
        try:
            wcs = ROB_2(x_opt)
            # assign outputs to easily readable variables (indexed by constraint number)
            lambdas_opt, LMI_vars_opt = [], []
            for i in range(0,len(inner_vars_list)*num_constraints,len(inner_vars_list)):
                lambdas_opt.append(wcs[i + 0]) # not needed
                LMI_vars_opt.append(wcs[i + 1])
            # add constraint penalty elemenst of objective
            loss_violations = 0

            eigenvalues_by_constraint = [torch.linalg.eig(LMI_vars_opt[nc]).eigenvalues.real for nc in range(num_constraints)]
            violations = []
            for nc, eigenvalues in enumerate(eigenvalues_by_constraint):
                # with negative lambdas this finds the least feasible solution, added relu as it should only contribute gradients when infeasible
                violation = torch.sum(torch.nn.ReLU()(-eigenvalues))
                violations.append(violation)
                loss_violations += violation

            loss += loss_violations
            # potentially add regularisation (eg penalise frobenius norm of matrix)
        except:
            violations = [torch.zeros(1) for nc in range(num_constraints)]



        if np.max([v.detach().numpy() for v in violations]) <= 0:
            if loss_p.detach().numpy() < last_feasible_objective:
#                 print(j, loss_p.detach().numpy())
                last_feasible_x = x_opt.detach().numpy().copy()
                last_feasible_objective = loss_p.detach().numpy().copy()
        
        losses.append(loss_p.detach().squeeze())
        
        #backwards pass
        loss.backward()
        optimiser.step()
        
    time_delta = time.process_time() - start
    print(last_feasible_objective[0][0],f"deterministic surrogate solution after {epochs} steps")
    deterministic_surrogate_best_value.append(last_feasible_objective[0][0])
    deterministic_surrogate_solving_times.append(time_delta)
    
    start = time.process_time()
    with torch.no_grad():
        x_opt, = ROB_1(*(torch_parameters))
    time_delta = time.process_time() - start
    deterministic_surrogate_test_time.append(time_delta)
    

Problem size 10 started at Mon Dec 11 19:17:48 2023
-0.19491230508067203 robust reformulation solution


  0%|          | 0/200 [00:00<?, ?it/s]

[[-0.1816036]] deterministic surrogate solution after 200 steps
Problem size 20 started at Mon Dec 11 19:18:30 2023
-0.570378751563001 robust reformulation solution


  0%|          | 0/200 [00:00<?, ?it/s]

[[-0.54018]] deterministic surrogate solution after 200 steps
Problem size 30 started at Mon Dec 11 19:20:37 2023
-0.5077417320777612 robust reformulation solution


...

  0%|          | 0/200 [00:00<?, ?it/s]

[[-0.5478601]] deterministic surrogate solution after 200 steps
Problem size 50 started at Mon Dec 11 19:32:33 2023
-0.6000229009136051 robust reformulation solution


/Users/egonpersak/miniconda3/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:235: UserWarning: Your problem has too many parameters for efficient DPP compilation. We suggest setting 'ignore_dpp = True'.
  warnings.warn(


  0%|          | 0/200 [00:00<?, ?it/s]

[[-0.56081784]] deterministic surrogate solution after 200 steps


In [36]:
optimal_values = pd.DataFrame({"problem_size":sizes,
                               "robust_optimal_value": robust_optimal_value,
                               "deterministic_surrogate": deterministic_surrogate_best_value
                              })

optimal_values[f"gap_after_{epochs}_runs"] = (optimal_values["robust_optimal_value"] - optimal_values["deterministic_surrogate"])/optimal_values["robust_optimal_value"]
optimal_values.round(4)


,problem_size,robust_optimal_value,deterministic_surrogate,gap_after_200_runs
0,10,-0.1949,-0.1816,0.0683
1,20,-0.5704,-0.5402,0.0529
2,30,-0.5077,-0.4809,0.0528
3,40,-0.5866,-0.5479,0.0660
4,50,-0.6000,-0.5608,0.0653


In [32]:
times = pd.DataFrame({"problem_size":sizes,
                      "robust_counterpart_time": robust_solving_times,
                      "surrogate_solving_time": deterministic_surrogate_solving_times,
                      "surrogate_test_time": deterministic_surrogate_test_time
                     })
times = times.set_index("problem_size")

times.round(4)

,robust_counterpart_time,surrogate_solving_time,surrogate_test_time
problem_size,,,
10,0.1588,173.5898,0.0140
20,1.1843,441.9937,0.0705
30,1.4984,666.2519,0.0920
40,2.0021,1251.3561,0.1717
50,2.2666,1767.9229,0.0133
